In [ ]:
import SimRunner as sr
#this class hides many details. to see everything that is happening you must study the SimRunner.py file as well as sus.#kyle_tools.multisim
bfr = sr.BitFlipRunner()

In [ ]:
plt.rcParams["animation.html"] = "jshtml"
from IPython.display import HTML
import matplotlib.animation as animation
%matplotlib inline

In [ ]:
#setting up parameters

p_vals={}
p_vals['lambda']=1
p_vals['N'] = 50_000
p_vals['dt'] = 1/50_000

p_vals['k'] = 64
#p_vals['tau'] = np.pi/np.sqrt(p_vals['k'])

p_vals['depth'] = 30
p_vals['location'] = 1
p_vals['verbose'] = False

bfr.change_params(p_vals)


In [ ]:
bfr.eq_system.potential.offset
#bfr.sim.system.potential.offset

In [ ]:
bfr.initialize_sim()
fig, ax = plt.subplots()
bfr.eq_system.show_potential(1, ax=ax)
bfr.sim.system.show_potential(1, ax=ax)
ax.set_xlim(-1.5,1.5)
ax.set_ylim(-50,50)
ax.axhline(0)

In [ ]:
%%time
bfr.run_sim(verbose=True)

In [ ]:
sum((bfr.eq_system.get_energy(bfr.sim.output.final_state,0))>0)

In [ ]:
works = []
final_e = []
ks = [9, 16, 25, 36, 49, 64, 81]
for item in ks:
    p_vals['k'] = item
    p_vals['tau'] = np.pi/np.sqrt(p_vals['k'])
    bfr.change_params(p_vals)
    bfr.run_sim(verbose=True)
    print(f'{item}')
    works.append(bfr.sim.output.final_W)
    final_e.append(bfr.eq_system.get_energy(bfr.sim.output.final_state,0))    

In [ ]:
w_avg = np.array([np.mean(item) for item in works])
w_std = np.array([np.std(item)/np.sqrt(p_vals['N']) for item in works])
j_avg = np.array([np.mean(np.exp(-item)) for item in works])
j_std = np.array([np.std(np.exp(-item))/np.sqrt(p_vals['N']) for item in works])
fid = [ sum( item>=0 )/len(item) for item in final_e]

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
ax.errorbar([np.pi/np.sqrt(k) for k in ks] , w_avg, yerr=3*w_std, marker='o', linestyle='--')
ax.set_xlabel('$\\tau$');
ax.set_ylabel('$\\langle W \\rangle (k_BT)$');
ax.set_title(f'damping fixed at {p_vals["lambda"]}, initial depth {p_vals["depth"]} kT')


In [ ]:
fig.savefig('l1p0.pdf')

In [ ]:
works = []
for i in range(100):
    bfr.run_sim(verbose=False);

    works.append(bfr.sim.output.final_W)
        

In [ ]:
w_avg = np.array([np.mean(item) for item in works])
w_std = np.array([np.std(item)/np.sqrt(p_vals['N']) for item in works])
j_avg = np.array([np.mean(np.exp(-item)) for item in works])
j_std = np.array([np.std(np.exp(-item))/np.sqrt(p_vals['N']) for item in works])

fig, ax = plt.subplots(1,2, figsize=(20,6))

ax[0].errorbar(range(100), w_avg, yerr=2*w_std, linestyle='none', marker='o')
for item in [-1,0,1]:
    ax[0].axhline(w_avg.mean()+item*2*w_avg.std())

ax[1].errorbar(range(100), j_avg, yerr=2*j_std, linestyle='none', marker='o')
for item in [-1,0,1]:
    ax[1].axhline(j_avg.mean()+item*2*j_avg.std())    

In [ ]:
# if you arent sure what attribites the sim output has, this is a useful trick:
bfr.sim.output.__dict__.keys()

In [ ]:
plt.hist(bfr.sim.output.final_W)
np.mean(np.exp(-bfr.sim.output.final_W))

In [ ]:
#here is a quick trajectory trace
allstate = bfr.sim.output.all_state['states']
plt.plot(allstate[:,:,0,0].transpose());


In [ ]:
# try to create a ParamGuider class from source.kyle_tools.multisim OR make your own similar class in order to 'optimize' this bit flip. By optimize, I mean to find a protocol that has low work but also high fidelity.  Keep lambda at .5 but feel free ro change the other parameters.

# It is important also to make sure that we have the right dt, so fist come up with with a method to determine how # small of a dt is necesarry to get good convergence and how many N we need to have good statistics.